This notebook takes an area of interest (Tampa Bay, FL) and a csv of water quality data (temperature). It starts by walking through how to fill that area of interest polygon with H3 Heaxagons of different sizes. It then walks through aggregating water quality data to a given scale of hexagons.

## Part 1: Area of Interest filled with different sized hexagons

In [1]:
# Import all the required libraries
import os
import pandas
import numpy
import geopandas
import h3
import matplotlib.pyplot as plt
from shapely.geometry import shape, mapping

%matplotlib notebook

In [2]:
# Functions for Part 1
def poly_hex_idx(gdf, hexagon_size):
    """ Polygon to hex by fill (only 1 polygon at a time)"""
    hex_col = 'hex' + str(hexagon_size)
    temp = mapping(gdf)  #shapely.mapping()
    temp_geo = temp['features'][0]['geometry']
    # Switch temp_geo x-y order and convert tuple to list
    coord = 'coordinates'
    temp_geo[coord] = [[(j[1], j[0]) for j in geo] for geo in temp_geo[coord]]
    # Create dataframe for polygon filled with hexagons
    hex_fill = pandas.DataFrame(h3.polyfill(temp_geo,
                                            hexagon_size),
                                columns=[hex_col])

    return hex_fill


def boundary_geom(hex_id):
    '''Implement h3_to_geo_boundary in bulk'''
    return shape({'type': 'Polygon',
                  'coordinates': [h3.h3_to_geo_boundary(h=hex_id,
                                                        geo_json=True)]})

Load Area of Interest (AOI) shapefile from temperature_data folder in current working directory.

In [3]:
data_dir = os.path.join(os.getcwd(), 'temperature_data')
# Original source:
shp_aoi = os.path.join(data_dir, 'TampaBay.shp')
gdf_aoi = geopandas.read_file(shp_aoi)
# Transform to WGS84 since that is what H3 boundaries are converted to
gdf_aoi.to_crs(epsg=4326, inplace=True)
# Plot the AOI
gdf_aoi.plot(edgecolor='k')

<IPython.core.display.Javascript object>

Create a geodataframe with the hexagon boundaries

In [4]:
# Set the desired hexagon size (1-15)
hex_size = 7
hex_column = 'hex' + str(hex_size)
# Create data frame of hex ids of the set size that fill the aoi
aoi_hex_fill = poly_hex_idx(gdf_aoi, hex_size)
# To map these hexagons they need planar coordinates
aoi_hex_fill['geom'] = aoi_hex_fill[hex_column].apply(lambda x: boundary_geom(x))
# The boundary coordinates can be used to create a geopandas dataframe
gdf_hex = geopandas.GeoDataFrame(aoi_hex_fill, geometry='geom')
# Read out average area of hexagons at this scale (project it to linear units=m)
print('Average area for scale {}: {} (range: {} to {})'
      .format(hex_size,
              gdf_hex['geom'].area.mean(),
              gdf_hex['geom'].area.min(),
              gdf_hex['geom'].area.max())
     )

Average area for scale 7: 0.00044188451355074037 (range: 0.0004402690775397927 to 0.000444279965562313)


In [5]:
#gdf_hex['geom'].to_crs(epsg=3857).area.mean()
#gdf_hex.crs

Plot the Tampa polygon filled with hexagons (use the interactive map to zoom in on small hexagons)

In [6]:
fig, ax = plt.subplots()
#ax=gdf_aoi.plot(edgecolor='k')
gdf_hex.plot(ax=ax, facecolor='none', edgecolor='red')

<IPython.core.display.Javascript object>

Geopandas can easily be saved as a shapefile (uncomment to save shape file)

In [7]:
#out_shp = os.path.join(data_dir, 'fill_{}.shp'.format(hex_column))
#gdf_hex.to_file(out_shp)

## Part 2: Load the example point data (downloads as two parts: points shp and csv data)

In [8]:
# Functions for Part 2
def wqp_stats(df, datetime_field, loc_field, statReportType='daily', statTypeCd='mean'):
    """
    NWIS-like daily statistics function for waterquality portal

    Parameters:
        statReportType (string): daily (default), monthly, or annual
        statTypeCd (string): all, mean, max, min, median

        Start and End will always be min/max timestamp in dataframe

    Returns:
        Dataframe of summary statistics
    """
    pd_freq = {'daily': 'D',
               'monthly': 'M',
               'annual': 'A'
               }
    if statReportType in pd_freq.keys():
        freq = pd_freq[statReportType]
    else:
        freq = statReportType

    # Loc_ID field to groupby (deal with depth later)
    #df['Loc_ID'] = df.index

    # Resample by set increment
    if statTypeCd == 'all':
        #return all stats as multiple columns, loop over self call?
        df_mean = wqp_stats(df, datetime_field, loc_field, statReportType)
        df_max = wqp_stats(df, datetime_field, loc_field, statReportType, 'max')
        #TODO: recombine
    elif statTypeCd == 'mean':
        df = df.groupby(loc_field).resample(freq, on=datetime_field).mean()
    elif statTypeCd == 'max':
        df = df.groupby(loc_field).resample(freq, on=datetime_field).max()
    elif statTypeCd == 'min':
        df = df.groupby(loc_field).resample(freq, on=datetime_field).min()
    elif statTypeCd == 'median':
        df = df.groupby(loc_field).resample(freq, on=datetime_field).median()
    #else: statTypeCd error?
    return df.reset_index()


def hex_idx(gdf, hexagon_size, method='count'):
    """ Point to hex by aggregating"""
    h3_idx_col = 'hex' + str(hexagon_size)
    # Assign hex ID
    gdf[h3_idx_col] = gdf.apply(lambda x: h3.geo_to_h3(x.geometry.y,
                                                       x.geometry.x,
                                                    hexagon_size), 1)
    # Aggregate into hexagons
    if method == 'avg':
        pnt_hex = gdf.groupby(h3_idx_col).mean()
    else:
        pnt_hex = gdf.groupby(h3_idx_col).size().to_frame('cnt')
    return pnt_hex.reset_index()

In [10]:
# WQP station points geojson
# From WQP: https://www.waterqualitydata.us/portal/ (see getWQP notebook for query)
json_loc = os.path.join(data_dir, 'AOI_Temperature_Results_DT.geojson')
gdf_temp = geopandas.read_file(json_loc)  # Read to geodataframe

DriverError: L:\Public\jbousqui\Code\GitHub\test_notebook\temperature_data\AOI_Temperature_Results_DT.geojson: No such file or directory

In [ ]:
# TODO: is an extra index field being added?

In [ ]:
# Recognize datetime fields
gdf_temp['Activity_datetime'] = pandas.to_datetime(gdf_temp['Activity_datetime'])
gdf_temp['Analysis_datetime'] = pandas.to_datetime(gdf_temp['Analysis_datetime'])
print('Date range: {} to {}'.format(min(gdf_temp['Activity_datetime']),
                                    max(gdf_temp['Activity_datetime'])))

In [ ]:
# Confirm unique sample sites in dataset
print('Sample sites in AOI: {}'.format(len(set(gdf_temp['MonitoringLocationIdentifier']))))

In [ ]:
# Create geodataframe with just geometry
geom = gdf_temp[['MonitoringLocationIdentifier', 'geometry']].drop_duplicates()

In [ ]:
# Read the csv to a df, dropping empty fields

# # Columns that are already in sample points table and not used for join
# dup_cols = ['OrganizationIdentifier', 'OrganizationFormalName', 'ProviderName']
# # Read csv to dataframe
# df_temp = pandas.read_csv(csv_temp,
#                           low_memory=False)
# # Drop missing data columns
# df_temp.dropna(axis=1, how='all', inplace=True)

# Join sample points and temperature results on 'Loc_ID'

# gdf_temp = gdf.set_index('Loc_ID').join(df_temp.set_index('MonitoringLocationIdentifier'))
# #NOTE: if renamed so both are Loc_ID the index will have a name
# gdf_temp.dropna(axis=1, how='all', inplace=True)  # Drop empty column again

In [ ]:
# # Temporary cell - check dates on EPC data
# epc = gdf_temp[gdf_temp.OrganizationFormalName == 'Environmental Protection Commission of Hillsborough County']
# print('Date range: {} to {}'.format(min(epc['Activity_datetime']), max(epc['Activity_datetime'])))

Datetime fields can be used to get specific periods of time and average across set increments (e.g. average daily value from hourly data)

In [ ]:
# Get daily averages
loc_ID = 'MonitoringLocationIdentifier'
df_day_all = wqp_stats(gdf_temp, 'Activity_datetime', loc_ID, statReportType='daily')
df_day_avg = df_day_all[df_day_all['deg_c'].notna()]
print('Days in range: {}'.format(len(df_day_all.groupby('Activity_datetime'))))
print('Daily results across sites: {}'.format(len(df_day_avg)))
print('Days with results at any site: {}'.format(len(df_day_avg.groupby('Activity_datetime'))))

In [ ]:
# Get annual averages
df_year_avg = wqp_stats(gdf_temp, 'Activity_datetime', loc_ID, statReportType='annual')
print('Years in range: {}'
      .format(len(df_year_avg.groupby('Activity_datetime').count())))
print('Annual results across sites: {}'
      .format(len(df_year_avg[df_year_avg['deg_c'].notna()])))
print('Locations with results in multiple years: {}'
      .format((df_year_avg.groupby(loc_ID)
                           .count()['Activity_datetime'] == 1).sum()))
annual_counts = df_year_avg.groupby('Activity_datetime').count()['deg_c']
max_year_df = annual_counts[annual_counts == max(annual_counts)].reset_index()
max_year = max_year_df['Activity_datetime'][0].year
print('{} was the year with the most results, {}'
      .format(max_year, max_year_df['deg_c'][0]))
# Same as gdf_temp.groupby(loc_ID).resample('A', on='Activity_datetime').mean()

In [ ]:
# Limit to max year
df_max = df_year_avg[df_year_avg['Activity_datetime'] == max_year_df['Activity_datetime'][0]]
# Aggregate on geom to get stations for max year -?Already done
# station_max = df_max.groupby(loc_ID).count()

# Join back to geometry
gdf_max = df_max.set_index(loc_ID).join(geom.set_index(loc_ID))
#gdf_station_max = station_max.join(geom.set_index(loc_ID))

Aggregate point temperatures to hex

In [ ]:
# Set the desired hexagon size (1-15)
hex_size = 7
hex_column = 'hex' + str(hex_size)
# Values within one hex can be aggregated multiple ways
station_hex = hex_idx(geom, hex_size)  # Count - stations 
df_hex = hex_idx(gdf_temp, hex_size)  # Count - data points
# Max year - add average & count to table

# Aggregate on geom to get stations for max year
#station_max_hex = hex_idx(gdf_station_max, hex_size)  # Count - stations
hex_max_cnt = hex_idx(gdf_max, hex_size)  # Count - data points
hex_max = hex_idx(gdf_max, hex_size, 'avg')  # Average temperature
# Combine (todo: move to function)
hex_max = hex_max.set_index(hex_column).join(hex_max_cnt.set_index(hex_column))
hex_max.reset_index(inplace=True)

In [ ]:
# Print info from tables
# Stations - all years
print('All results:')
print('Number of stations per hexagon: {} to {}, {} average'
      .format(min(station_hex['cnt']),
              max(station_hex['cnt']),
              station_hex['cnt'].mean()))
print('There are {} hexagons with {} stations'
      .format(len(station_hex[station_hex['cnt']==max(station_hex['cnt'])]),
              max(station_hex['cnt'])))
# Results - all years
print('Hexagons with temperature data: {}'.format(len(set(df_hex[hex_column]))))
# Results - max year
print('\nMax year results:')
print('Number of stations per hexagon: {} to {}, {} average'
      .format(min(hex_max['cnt']),
              max(hex_max['cnt']),
              hex_max['cnt'].mean()))
print('There are {} hexagons with {} stations'
      .format(len(hex_max[hex_max['cnt']==max(hex_max['cnt'])]),
              max(hex_max['cnt'])))
print('Hexagons with temperature data: {}'.format(len(set(hex_max[hex_column]))))

Add geometry back to the dataframe to map it

In [ ]:
# Add geometry to hexagon dataframes
station_hex['geom'] = station_hex[hex_column].apply(lambda x: boundary_geom(x))
gdf_station_hex = geopandas.GeoDataFrame(station_hex, geometry='geom')

df_hex['geom'] = df_hex[hex_column].apply(lambda x: boundary_geom(x))
gdf_hex = geopandas.GeoDataFrame(df_hex, geometry='geom')

# Annually averaged stations to hex
hex_max['geom'] = hex_max[hex_column].apply(lambda x: boundary_geom(x))
gdf_max_hex = geopandas.GeoDataFrame(hex_max, geometry='geom')

In [ ]:
gdf_hex.plot(column='cnt', cmap='Blues', legend=True)

In [ ]:
gdf_max_hex.plot(column='deg_c', cmap='OrRd', legend=True)

Geopandas can easily be saved as a shapefile (uncomment to save shape file)

In [ ]:
# # All hexagons
# out_hex_stations_shp = os.path.join(data_dir, '{}_stations_all.shp'.format(hex_column))
# gdf_station_hex.dropna(axis=1, how='all', inplace=True)
# gdf_station_hex.to_file(out_hex_stations_shp)

# out_hex_shp = os.path.join(data_dir, '{}_avg_all.shp'.format(hex_column))
# gdf_hex.dropna(axis=1, how='all', inplace=True)
# gdf_hex.to_file(out_hex_shp)

# Max year
out_max_year_shp = os.path.join(data_dir, '{}_avg_{}.shp'.format(hex_column, max_year))
gdf_max_hex.dropna(axis=1, how='all', inplace=True)
gdf_max_hex.to_file(out_max_year_shp)

## Part 3: Interpolate areas where temperatures are missing

In [ ]:
def kring_IDW(df, hex_col, metric_col, k_max):
    '''
    Uses hollow rings method, solid disks may be faster
    k_max is the number of rings out from a cell to consider
    '''
    df_agg = df[[hex_col]]
    df_agg['hexk'] = df_agg[hex_col]
    df_agg.set_index(hex_col, inplace=True)
    temp2 = [df_agg['hexk'].reset_index()]
    temp2[-1]['k'] = 0

    for k in range(1, k_max):
        temp2.append((df_agg['hexk']
                      .apply(lambda x: pandas.Series(list(h3.hex_ring(x, k)))).stack()
                      .to_frame('hexk').reset_index(1, drop=True).reset_index()
                     ))
        temp2[-1]['k'] = k
    df_all = pandas.concat(temp2).merge(df)
    df_all.dropna(subset=[metric_col], inplace=True)  # Drop NaN
    # Instead of applying the coef, divide by k
    #df_all[metric_col] = df_all[metric_col] * df_all.k.apply(lambda x:coef[x])
    df_all['num_val'] = df_all[metric_col]/df_all.k  # if k=0, x/k = inf
    # sum numerator
    dfs_t = df_all.groupby('hexk')[['num_val']].sum()
    # sum denominator
    df_all['den_val'] = 1.0/df_all.k
    #nan -> inf so convert back and drop nan
    df_all['den_val'] = df_all['den_val'].replace(numpy.inf, numpy.nan)
    #df_all.dropna(subset=['num_val'], inplace=True)
    dfs_b = df_all.groupby('hexk')[['den_val']].sum()

    dfs = dfs_t.join(dfs_b)
    dfs['idw'] = dfs['num_val']/dfs['den_val']
    #reduce dfs
    dfs = dfs[['idw']]
    #fix index name
    dfs.index.rename(hex_col, inplace=True)
    #join back to df
    dfs = df.set_index(hex_col).join(dfs)
    #replace inf with orig?
    return dfs.reset_index()

If running part 3 independently re-define these variables

In [ ]:
hex_size = 7
hex_column = 'hex' + str(hex_size)
# Original source:
data_dir = os.path.join(os.getcwd(), 'temperature_data')
shp_aoi = os.path.join(data_dir, 'TampaBay.shp')
gdf_aoi = geopandas.read_file(shp_aoi)
gdf_aoi.to_crs(epsg=4326, inplace=True)  # Transform to WGS84
# Create filled area (define where temperatures are wanted)
aoi_hex_fill = poly_hex_idx(gdf_aoi, hex_size)
# Dataframe where temperatures are known (e.g. max year)
out_max_year_shp = os.path.join(data_dir, 'hex7_avg_2004.shp')
gdf_max_hex = geopandas.read_file(out_max_year_shp)

Combine aoi_hex_fill from part 1 and gdf_max_hex from part 2

In [ ]:
# Combine hex data and filled hex
df_comb = aoi_hex_fill.set_index(hex_column).join(gdf_max_hex.set_index(hex_column))
df_comb.reset_index(inplace=True)

In [ ]:
# Fill missing cells using IDW
df_max_idw = kring_IDW(df_comb, hex_column, metric_col='deg_c', k_max=20)
# Add geometry to hexagons to map
df_max_idw['geom'] = df_max_idw[hex_column].apply(lambda x: boundary_geom(x))
gdf_max_idw = geopandas.GeoDataFrame(df_max_idw, geometry='geom')

In [ ]:
#check kring_IDW for something being done on a slice instead of the df

In [ ]:
gdf_max_idw.plot(column='idw', cmap='OrRd', legend=True)

In [ ]:
# Save interpolated surface
idw_hex_shp = os.path.join(data_dir, 'idw_7_2004_temp.shp')
gdf_max_idw.drop(columns=['USGSPCode','geometry'], inplace=True)
gdf_max_idw.to_file(idw_hex_shp)